# Trends

In [1]:
include("stuff.jl")

Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /buildworker/worker/package_linux64/build/src/locks.h:138 [inlined]
jl_typeinf_end at /buildworker/worker/package_linux64/build/src/gf.c:2610
typeinf_ext at ./compiler/typeinfer.jl:577
typeinf_ext at ./compiler/typeinfer.jl:613
jfptr_typeinf_ext_1.clone_1 at /Net/Groups/BGI/people/gkraemer/progs/julia/julia-1.1.0-bin/lib/julia/sys.so (unknown line)
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2219
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1571 [inlined]
jl_type_infer at /buildworker/worker/package_linux64/build/src/gf.c:277
jl_compile_method_internal at /buildworker/worker/package_linux64/build/src/gf.c:1819 [inlined]
jl_fptr_trampoline at /buildworker/worker/package_linux64/build/src/gf.c:1863
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2219
do_call at /buildworker/worker/package_linux64/build/src/interpreter.c:32

LoadError: SystemError (with /home/science/summarizing_biosphere/results): mkdir: No such file or directory

In [ ]:
pca_trends = loadCube(joinpath(cubes_base_dir, "pca_trends"))

In [ ]:
pca_trends_pvals_adj = loadCube(joinpath(cubes_base_dir, "pca_trends_pvals_adj"))

In [ ]:
pca_trends_pvals = loadCube(joinpath(cubes_base_dir, "pca_trends_pvals"))

In [ ]:
#pt_trends = loadCube(joinpath(cubes_base_dir, "pt_trends"))
#pca_trends_2 = loadCube(joinpath(cubes_base_dir, "pca_trends_2"))
#pt_trends_2 = loadCube(joinpath(cubes_base_dir, "pt_trends_2"))
#dryness_cube2 = loadCube(joinpath(cubes_base_dir, "dryness_cube2"))
#dryness_cube1 = loadCube(joinpath(cubes_base_dir, "dryness_cube1"))

In [ ]:
ifelsemiss(c::Union{Missing, Bool}, t, f, m = missing) = 
if ismissing(c) 
    return m
elseif c
    return t
elseif !c
    return f
else
    error("something is wrong")
end

In [ ]:
R"""
pdf($(joinpath(fig_path, "slopes_pc12_nolog2.pdf")), width = 14, height = 5)
legend_size <- 120
legend_x <- unit(legend_size / 2, "points") + unit(2.9, "line")
legend_y <- unit(legend_size / 2, "points") + unit(4, "line")
axis_label_rot <- 20
ax_lab_cex <- 0.7

# the slopes are per 8 days, we have to switch to  per year
map_data <- $(colorbar_2d_teuling2011_quant(
        ((365 / 8) .* pca_trends[1, :, :] .* (pca_trends_pvals_adj[1, :, :] .< 0.05)),
        ((365 / 8) .* pca_trends[2, :, :] .* (pca_trends_pvals_adj[2, :, :] .< 0.05)),
#ifelsemiss.(pca_trends_pvals_adj[1, :, :] .< 0.05, (365 / 6) .* pca_trends[1, :, :], zero(eltype(pca_trends))),
#ifelsemiss.(pca_trends_pvals_adj[2, :, :] .< 0.05, (365 / 6) .* pca_trends[2, :, :], zero(eltype(pca_trends))),
#        ((365 / 8) .* pca_trends[1, :, :]),
#        ((365 / 8) .* pca_trends[2, :, :]),
#        get_q((365 / 8) .* (pca_trends_pvals_adj[1, :, :] .< 0.05) .* pca_trends[1, :, :], 0.1),
#        get_q((365 / 8) .* (pca_trends_pvals_adj[2, :, :] .< 0.05) .* pca_trends[2, :, :], 0.1)
        get_q((365 / 8) .* pca_trends[1, :, :], 0.1),
        get_q((365 / 8) .* pca_trends[2, :, :], 0.1),
        "#F0F0F0"
))

print("hello")

x_ticks <- map_data$x_ticks
y_ticks <- map_data$y_ticks

pushViewport(viewport(
    x = unit(5, "inches"),
    y = unit(2.5, "inches"),
    width = unit(10 / 14, "npc"),
    height = unit(1, "npc")
))
grid.raster(
    t(map_data$col_map),
    interpolate = FALSE
)

popViewport()
pushViewport(viewport(
    x = legend_x, 
    y = legend_y, 
    width = unit(legend_size, "points"), 
    height = unit(legend_size, "points"),))


grid.raster(
    t($(colorbar_2d_teuling2011_quant(
        reshape(repeat(-2:2, inner = 5), 5, 5)',
        reshape(repeat(2:-1:-2, inner = 5), 5, 5), 
        (-2, 2), (-2, 2)
    )[:col_map])),
    interpolate = FALSE
)

grid.xaxis(
    at = seq(from = 0, to = 1, length.out = length(x_ticks)),
    label = c(" ", label = sprintf("%0.2e", x_ticks)[c(-1, -length(x_ticks))], " "),
    gp = gpar(cex = ax_lab_cex),
    edits = gEdit(gPath = "labels", rot = axis_label_rot, just = "top")
)
grid::grid.yaxis(
    at = seq(from = 0, to = 1, length.out = length(y_ticks)),
    label = c(" ", label = sprintf("%0.2e", y_ticks)[c(-1, -length(y_ticks))], " "),
    gp = gpar(cex = ax_lab_cex),
    edits = gEdit(gPath = "labels", rot = 90 - axis_label_rot, just = "bottom")
)
grid::grid.text(
    expression(lower%<-%PC[1]~group("[", scriptstyle(frac(std, y)), "]")%->%higher),
    x = 0.5, 
    y = unit(0, "npc") - unit(1.9, "line"),
    just = "top"
)
grid::grid.text(
    "productivity",
    x = 0.5, 
    y = unit(0, "npc") - unit(3, "line"),
    just = "top"
)
grid::grid.text(
    expression(wetter%<-%PC[2]~group("[", scriptstyle(frac(std, y)), "]")%->%dryer),
    x = unit(-1.7, "line"),
    y = 0.5,
    just = "bottom", rot = 90
)


#legend_data <- $(colorbar_2d_teuling2011_quant(
#        ((365 / 8) .* pca_trends[1, :, :] .* (pca_trends_pvals_adj[1, :, :] .< 0.05)),
#        ((365 / 8) .* pca_trends[2, :, :] .* (pca_trends_pvals_adj[2, :, :] .< 0.05)),
#ifelsemiss.(pca_trends_pvals_adj[1, :, :] .< 0.05, (365 / 6) .* pca_trends[1, :, :], zero(eltype(pca_trends))),
#ifelsemiss.(pca_trends_pvals_adj[2, :, :] .< 0.05, (365 / 6) .* pca_trends[2, :, :], zero(eltype(pca_trends))),
#        ((365 / 8) .* pca_trends[1, :, :]),
#        ((365 / 8) .* pca_trends[2, :, :]),
#        get_q((365 / 8) .* (pca_trends_pvals_adj[1, :, :] .< 0.05) .* pca_trends[1, :, :], 0.1),
#        get_q((365 / 8) .* (pca_trends_pvals_adj[2, :, :] .< 0.05) .* pca_trends[2, :, :], 0.1)
#        get_q((365 / 8) .* pca_trends[1, :, :], 0.1),
#        get_q((365 / 8) .* pca_trends[2, :, :], 0.1)
#))

#pca_col <- as.vector(legend_data$col_map)

pca_1_coords <- $(((365 / 8) .* pca_trends[1, :, :]) |> collect |> missing_to_nan |> vec)
pca_2_coords <- $(((365 / 8) .* pca_trends[2, :, :]) |> collect |> missing_to_nan |> vec)

pca_col <- as.vector(map_data$col_map)

#pca_1_coords <- $(((365 / 8) .* pca_trends[1, :, :] .* (pca_trends_pvals_adj[1, :, :] .< 0.05)) |> collect |> missing_to_nan |> vec)
#pca_2_coords <- $(((365 / 8) .* pca_trends[2, :, :] .* (pca_trends_pvals_adj[2, :, :] .< 0.05)) |> collect |> missing_to_nan |> vec)

popViewport()
pushViewport(viewport(
    x = unit(12.5, "inches"), 
    y = unit(2.5, "inches"), 
    width = unit(3, "inches"), 
    height = unit(3, "inches"),
    xscale = range(pca_1_coords, na.rm = TRUE),
    yscale = range(pca_2_coords, na.rm = TRUE)
))

grid.points(
    x = pca_1_coords,
    y = pca_2_coords,
    pch = ".",
    gp = gpar(col = pca_col)
)
grid.xaxis(gp = gpar(cex = ax_lab_cex))
grid.yaxis(gp = gpar(cex = ax_lab_cex))
grid.text(expression(lower%<-%PC[1]~group("[", scriptstyle(frac(std, y)), "]")%->%higher), 0.5, unit(-2.5, "lines"))
grid.text(expression(wetter%<-%PC[2]~group("[", scriptstyle(frac(std, y)), "]")%->%dryer), unit(-3, "lines"), 0.5, rot = 90)

popViewport()

grid.text("(a)", 0,     1, hjust = 0, vjust = 1)
grid.text("(b)", 10/14, 1, hjust = 0, vjust = 1)

dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/slopes_pc12_nolog2.pdf", width = 900)

In [ ]:
R"library(hexbin)"

In [ ]:
R"""
pdf($(joinpath(fig_path, "slopes_pc123.pdf")), width = 7, height = 8)
data(coastsCoarse, package = "rworldmap")

wds <- c(1, 0.7)
hts <- c(1, 0.3)

layout(
    matrix(
        c(
            1, 7,
            2, 7,
            3, 8,
            4, 8,
            5, 9,
            6, 9
        ), byrow = TRUE, ncol = 2
    ),
    heights = rep(hts, times = 3),
    widths = wds
)

pal1 <- c("#543005", "#8C510A", "#BF812D", "#DFC27D", 
          "#F6E8C3", "#F5F5F5", "#C7EAE5", "#80CDC1", 
          "#35978F", "#01665E", "#003C30")
pal1 <- colorRampPalette(colors = pal1)(100)

pal2 <- rev(c("#67001f", "#b2182b","#d6604d", "#f4a582",
              "#fddbc7", "#f7f7f7", "#d1e5f0","#92c5de",
              "#4393c3", "#2166ac","#053061"))
pal2 <- colorRampPalette(colors = pal2)(100)

pal3 <- c("#7f3b08","#b35806","#e08214","#fdb863",
          "#fee0b6","#f7f7f7","#d8daeb","#b2abd2",
          "#8073ac","#542788","#2d004b")
pal3 <- colorRampPalette(colors = pal3)(100)

pc1_lab <- expression(lower%<-%PC[1]~group("[", scriptstyle(frac(std, y)), "]")%->%higher)
pc2_lab <- expression(wetter%<-%PC[2]~group("[", scriptstyle(frac(std, y)), "]")%->%dryer)
pc3_lab <- expression(darker%<-%PC[3]~group("[", scriptstyle(frac(std, y)), "]")%->%brighter)

lat_lims <- 60:600
lat_lims_deg <- rev( $(getAxis(LatAxis, pca_trends).values |> collect)[lat_lims] )
lon_lims <- 1:1440
lon_lims_deg <- $(getAxis(LonAxis, pca_trends).values |> collect)[lon_lims]

r2a <- function(x, lat_limst = lat_lims) {
    x <- as.array(x)
    x <- x[lat_lims, ]
    x <- t(x)[,dim(x)[1]:1]
}

map1 <- r2a(t($(pca_trends[1, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)))
map2 <- r2a(t($(pca_trends[2, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)))
map3 <- r2a(t($(pca_trends[3, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)))

pmap1 <- r2a(t($(pca_trends_pvals_adj[1, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)))
pmap2 <- r2a(t($(pca_trends_pvals_adj[2, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)))
pmap3 <- r2a(t($(pca_trends_pvals_adj[3, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)))


lim_map <- function(map) {
    res <- range(map, na.rm = TRUE)
    res <- max(abs(res))
    res <- c(-res, res)
}

plot_map <- function(map, pvals, col, lons = lon_lims_deg, lats = lat_lims_deg) {
    par(mar = c(0, 0, 0, 0))
    map[pvals >= 0.05] <- 0
    image(
        x = lons,
        y = lats,
        z = map, 
        col = col, 
        useRaster = TRUE,
        zlim = lim_map(map),
        bty = "n",
        xlab = NA, ylab = NA,
        axes = FALSE
    )
    plot(coastsCoarse, add = TRUE)
}

plot_bar <- function(map, col, xlab) {
    par(mar = c(4, 4, 0, 4) + 0.1)
    val_lims <- lim_map(map)
    image(
        x = seq(val_lims[1], val_lims[2], length.out = 101),
        y = 1,
        z = matrix(1:100),
        col = col,
        yaxt = "n",
        bty = "n",
        xlab = xlab,
        ylab = NA,
        useRaster = TRUE
    )
}

`/.unit` <- function(x, y) {
    x2 <- as.numeric(convertUnit(x, "pt"))
    res <- x2 / y
    unit(res, "pt")
}

plot_scatter <- function(map1, map2, xlab = NA, ylab = NA) {
    par(mar = c(4, 4, 0, 0) + 0.1, xpd = NA)
    #plot(as.vector(map1), as.vector(map2), col = "#00000011", pch = ".", ...)

    ##### advance layout
    plot.new()
    pf <- par()$fig
    pm <- par()$mar

    ##### calculations for the new viewport
    w <- unit(pf[2], "npc") - unit(pf[1], "npc") - unit(pm[2], "lines") - unit(pm[4], "lines")
    h <- unit(pf[4], "npc") - unit(pf[3], "npc") - unit(pm[1], "lines") - unit(pm[3], "lines")
    x <- unit(pf[1], "npc") + unit(pm[2], "lines") + w / 2
    y <- unit(pf[3], "npc") + unit(pm[1], "lines") + h / 2

    xscale <- lim_map(map1)
    yscale <- lim_map(map2)

    ##### the actual plotting
    hb <- hexbin(as.vector(map1), as.vector(map2))

    vp <- viewport(
        x = x, y = y, width = w, height = h, xscale = xscale, yscale = yscale,
        clip = FALSE
    )
    pushViewport(vp)
    grid.hexagons(hb)
    grid.xaxis(gp = gpar(cex = 0.7))
    grid.yaxis(gp = gpar(cex = 0.7))
    grid.text(xlab, 0.5, unit(-3, "lines"), gp = gpar(cex = 0.7))
    grid.text(ylab, unit(-4, "lines"), 0.5, gp = gpar(cex = 0.7), rot = 90)
 


    popViewport()
}


########################################## plot start

plot_map(map1, pmap1, col = pal1)
plot_bar(map1, col = pal1, xlab = pc1_lab)
plot_map(map2, pmap2, col = pal2)
plot_bar(map2, col = pal2, xlab = pc2_lab)
plot_map(map3, pmap2, col = pal3)
plot_bar(map3, col = pal3, xlab = pc3_lab)

plot_scatter(map1, map2, xlab = pc1_lab, ylab = pc2_lab)
plot_scatter(map1, map3, xlab = pc1_lab, ylab = pc3_lab)
plot_scatter(map2, map3, xlab = pc2_lab, ylab = pc3_lab)

grid.text("(a)", 0,                 1,   just = c(0, 1))
grid.text("(b)", wds[1] / sum(wds), 1,   just = c(0, 1))
grid.text("(c)", 0,                 2/3, just = c(0, 1))
grid.text("(d)", wds[1] / sum(wds), 2/3, just = c(0, 1))
grid.text("(e)", 0,                 1/3, just = c(0, 1))
grid.text("(f)", wds[1] / sum(wds), 1/3, just = c(0, 1))


dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/slopes_pc123.pdf", width = 700, height = 900)

In [ ]:
plotMAP(pca_trends, dmin = -0.005, dmax = 0.005)

In [ ]:
cube_base = getCubeData(Cube(data_cube_dir), time = (Date("2001-01-01"), Date("2011-12-31")))

In [ ]:
plotTS(cube_base, lat = 1.5, lon = 18.5, variable = "precipitation")

In [ ]:
function month_mean_ts(m, v)
    amj_idx = 
        cube_base.timeAxis.values |> 
        x -> findall(xx -> Month(xx) in [Month.(m)...], x)

    amj_y =
        cube_base.timeAxis.values[amj_idx] |>
        x -> Year.(x)

    amj_gpp = cube_base[
            axVal2Index(cube_base.lonAxis, 14):axVal2Index(cube_base.lonAxis, 30),
            axVal2Index(cube_base.latAxis,  5):axVal2Index(cube_base.latAxis, -6),
            :,
            axVal2Index(cube_base.varAxis, v)
        ] 
    
    amj_gpp = [
        mean(amj_gpp[:, :, amj_idx[amj_y .== y]], dims = 3)
        for y in unique(amj_y)
    ] |> x -> cat(x..., dims = 3)
    
    function slope(y)
        yy = missing_to_nan(y)
        x = [collect(1:length(y)) repeat([1], length(y))]
        #@show size(x)
        #@show x' * x
        #@show x' * y
        res = (x' * x) \ (x' * yy)
        return res[1]
    end
    
    #return mapslices(xx -> xx |> skipmissing |> mean, amj_gpp, dims = [3]) |> x -> dropdims(x, dims = 3)
    return mapslices(slope, amj_gpp, dims = [3]) |> x -> dropdims(x, dims = 3) |> x -> x[:, end:-1:1]
end

R"""

pdf($(joinpath(fig_path, "congo_browning.pdf")), width = 5, height = 4)
pal <- (brewer.pal(name = "BrBG", n = 11))
dat <- $(month_mean_ts(4:6, "gross_primary_productivity"))
sym_range <- function(d) {
    r <- range(d, na.rm = TRUE)
    ra <- abs(r)
    ram <- max(ra)
print(ram)
    c(-ram, ram)
}


image(
    seq(14, 30, length.out = dim(dat)[1]),
    seq(-6, 5, length.out = dim(dat)[2]),
    dat,
main = "GPP trends AMJ 2000 - 2011",
    zlim = sym_range(dat),
    col = pal,
    xlab = expression("Lon " * group("[", degree, "]")),
    ylab = expression("Lat " * group("[", degree, "]"))
)

plot(getMap(), add = TRUE)

image(t(1:11), col = pal)

dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/congo_browning.pdf", width = 900)
    

In [ ]:
R"library(rworldmap)"

In [ ]:
cube_base.varAxis.values

In [ ]:
myequal(x, y) = if ismissing(x) && ismissing(y)
    return true
elseif ismissing(x) && !ismissing(y)
    return false
elseif !ismissing(x) && ismissing(y)
    return false
else
    return x == y
end

In [ ]:
findall(!,
    myequal.(
        ifelsemiss.(pca_trends_pvals_adj[1, :, :] .< 0.05, (365 / 8) .* pca_trends[1, :, :], zero(eltype(pca_trends)), missing),
        ((365 / 8) .* pca_trends[1, :, :] .* (pca_trends_pvals_adj[1, :, :] .< 0.05))
    )    
)

In [ ]:
col = [
    "#F58439" "#FDB739" "#BED85A" "#76C04B" "#00AC4E";
    "#F47C57" "#FAB380" "#D5E6A2" "#81C99B" "#00B189";
    "#F1687A" "#F6A3AE" "#F0F0F0" "#77CDCE" "#00B4B7";
    "#EE4498" "#C880B6" "#A8A4D1" "#44B5E8" "#00B3E5";
    "#A53F97" "#7E54A2" "#596AB2" "#1977BD" "#007EC4"
    ] |>
    x -> x[:, end:-1:1] |>
    collect

scale_1_5(xx, xxlim) =
    ((xx - xxlim[1]) / (xxlim[2] - xxlim[1]) * 5) |>
    xxx -> ceil(Int, xxx) |>
    xxx -> clamp(xxx, 1, 5)

xlim = (-0.04, 0.04)
ylim = (-0.05, 0.05)

function f(x, y)
    if ismissing(x) || ismissing(y)
        missing
    elseif x == 0 && y == 0
        # the actual zeros are the nonsignificant values
        nonsignificant_color
    else
        i = scale_1_5(x, xlim)
        j = scale_1_5(y, ylim)
        col[i, j]
    end
end



In [ ]:
    (365 / 8) * pca_trends[1, 7, 75], 
    (365 / 8) * pca_trends[2, 7, 75],
    (365 / 8) * pca_trends[1, 7, 75] * (pca_trends_pvals_adj[1, 7, 75] < 0.05), 
    (365 / 8) * pca_trends[2, 7, 75] * (pca_trends_pvals_adj[2, 7, 75] < 0.05)


In [ ]:
f(
    (365 / 8) * pca_trends[1, 7, 75], 
    (365 / 8) * pca_trends[2, 7, 75]
),
f(
    (365 / 8) * pca_trends[1, 7, 75] * (pca_trends_pvals_adj[1, 7, 75] < 0.05), 
    (365 / 8) * pca_trends[2, 7, 75] * (pca_trends_pvals_adj[2, 7, 75] < 0.05)
)


#pca_trends_pvals_adj[1, 341, 71], pca_trends_pvals_adj[2, 341, 71]

In [ ]:
legend_data = colorbar_2d_teuling2011(
        ((365 / 8) .* pca_trends[1, :, :] .* (pca_trends_pvals_adj[1, :, :] .< 0.05)),
        ((365 / 8) .* pca_trends[2, :, :] .* (pca_trends_pvals_adj[2, :, :] .< 0.05)),
#ifelsemiss.(pca_trends_pvals_adj[1, :, :] .< 0.05, (365 / 8) .* pca_trends[1, :, :], zero(eltype(pca_trends))),
#ifelsemiss.(pca_trends_pvals_adj[2, :, :] .< 0.05, (365 / 8) .* pca_trends[2, :, :], zero(eltype(pca_trends))),
#        ((365 / 8) .* pca_trends[1, :, :]),
#        ((365 / 8) .* pca_trends[2, :, :]),
#        get_q((365 / 8) .* (pca_trends_pvals_adj[1, :, :] .< 0.05) .* pca_trends[1, :, :], 0.1),
#        get_q((365 / 8) .* (pca_trends_pvals_adj[2, :, :] .< 0.05) .* pca_trends[2, :, :], 0.1)
        (-0.04, 0.04), 
        (-0.05, 0.05)
)

In [ ]:
map_data = colorbar_2d_teuling2011(
        #((365 / 8) .* pca_trends[1, :, :] .* (pca_trends_pvals_adj[1, :, :] .< 0.05)),
        #((365 / 8) .* pca_trends[2, :, :] .* (pca_trends_pvals_adj[2, :, :] .< 0.05)),
#ifelsemiss.(pca_trends_pvals_adj[1, :, :] .< 0.05, (365 / 8) .* pca_trends[1, :, :], zero(eltype(pca_trends))),
#ifelsemiss.(pca_trends_pvals_adj[2, :, :] .< 0.05, (365 / 8) .* pca_trends[2, :, :], zero(eltype(pca_trends))),
        ((365 / 8) .* pca_trends[1, :, :]),
        ((365 / 8) .* pca_trends[2, :, :]),
#        get_q((365 / 8) .* (pca_trends_pvals_adj[1, :, :] .< 0.05) .* pca_trends[1, :, :], 0.1),
#        get_q((365 / 8) .* (pca_trends_pvals_adj[2, :, :] .< 0.05) .* pca_trends[2, :, :], 0.1)
        (-0.04, 0.04), 
        (-0.05, 0.05)
)

In [ ]:
myeq(x, y) = 
if ismissing(x) && ismissing(y)
    return true
elseif ismissing(x) && !ismissing(y)
    return false
elseif !ismissing(x) && ismissing(y)
    return false
elseif x == "#FFFFFF" || y == "#FFFFFF"
    return true
else
    return x == y
end

In [ ]:
findall(.!myeq.(legend_data, map_data))

In [ ]:
legend_data[75, 7]

In [ ]:
map_data[75, 7]

In [ ]:
pca_trends[1, 341, 71], pca_trends[2, 341, 71],
pca_trends_pvals_adj[1, 341, 71], pca_trends_pvals_adj[2, 341, 71]

In [ ]:
get_q((365 / 8) .* (pca_trends_pvals_adj[1, :, :] .< 0.05) .* pca_trends[1, :, :], 0.1),
  get_q((365 / 8) .* (pca_trends_pvals_adj[2, :, :] .< 0.05) .* pca_trends[2, :, :], 0.1),
  get_q((365 / 8) .* pca_trends[1, :, :], 0.1),
  get_q((365 / 8) .* pca_trends[2, :, :], 0.1)

In [ ]:
nomissidx = findall(!ismissing, pca_trends_pvals_adj[1, :, :] |> vec |> collect)
sp = sortperm(pca_trends_pvals_adj[1, :, :] |> vec |> collect |> x -> x[nomissidx])
spca = pca_trends[1, :, :] |> vec |> collect |> x -> x[nomissidx] |> x -> x[sp]
spcap = pca_trends_pvals[1, :, :] |> vec |> collect |> x -> x[nomissidx] |> x -> x[sp]

R"""
plot($spcap, type = "l")
plot(abs($spca), type = "l")
"""

In [ ]:
R"""
dryness_cube2 <- $(dryness_cube2[:, :] |> collect)
image((log10(dryness_cube2) > -0.16)[, dim(dryness_cube2)[2]:1])
""";

In [ ]:
R"""
# from rubel et al 2017:
koeppen_levels <- c('Af', 'Am', 'As', 'Aw', 
                    'BSh', 'BSk', 'BWh', 'BWk', 
                    'Cfa', 'Cfb','Cfc', 'Csa', 'Csb', 'Csc', 'Cwa','Cwb', 'Cwc', 
                    'Dfa', 'Dfb', 'Dfc','Dfd', 'Dsa', 'Dsb', 'Dsc', 'Dsd','Dwa', 'Dwb', 'Dwc', 'Dwd', 
                    'EF','ET', 'Ocean')
koeppen_classes <- t(as.matrix(aggregate(raster("/scratch/gkraemer/global_pca_j1/Map_KG-Global/KG_1986-2010.grd"), fact = 3, fun = modal)))
dim(koeppen_classes)
koeppen_lv1_classes <- apply(
    koeppen_classes, 
    1:2, 
    function(i) {
        tmp <- substr(koeppen_levels[i], 1, 1)
        if (tmp == "O") NA else tmp
    }
)
image((koeppen_lv1_classes == "A")[,dim(koeppen_lv1_classes)[2]:1])
"""

In [ ]:
R"""
dryness_cube1 <- $(dryness_cube1[:, :] |> collect)
dryness_cube2 <- $(dryness_cube2[:, :] |> collect)
image((log10(dryness_cube1) > 0.5)[, dim(dryness_cube1)[2]:1])
image((log10(dryness_cube2) > -0.16)[, dim(dryness_cube2)[2]:1])
""";

In [ ]:
R"""
landcover <- nc_open("/Net/Groups/BGI/data/DataStructureMDI/DATA/grid/Global/0d25_static/globland30/v1/Data/globland30.fractional.1440.720.nc")
landcover_layers <- lapply(names(landcover$var), function(x) ncvar_get(landcover, x))
landcover_array <- do.call(c, landcover_layers)
dim(landcover_array) <- c(1440, 720, length(landcover_layers))
max_landcover_array <- apply(landcover_array, c(1, 2), function(x) ifelse(all(is.na(x)), 0, which.max(x)))
landcover_class_names <- names(landcover$var)
image((max_landcover_array == 3)[,dim(max_landcover_array)[2]:1])
print(landcover_class_names)
print(dim(max_landcover_array))
max_landcover_array_classes <- apply(
    max_landcover_array, 
    1:2, 
    function(x) if(x == 0) NA else landcover_class_names[x]
)
image(max_landcover_array_classes == "Forest")
"""

In [ ]:
plotMAP(pca_trends)

In [ ]:
plotMAP(pca_trends_2)